In [ ]:
# Install the Kaggle library to fetch datasets directly from Kaggle
!pip install kaggle

# Configure Kaggle API key for authentication
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json  # Secure the API key

### Step 1: Importing the Face Mask Dataset

In [ ]:
# Use the Kaggle API to download the face mask dataset
!kaggle datasets download -d omkargurav/face-mask-dataset

# Unzip the downloaded dataset
from zipfile import ZipFile
dataset = '/content/face-mask-dataset.zip'

with ZipFile(dataset, 'r') as zip:
    zip.extractall()  # Extract contents to the current directory
    print('The dataset has been extracted')

# Verify the extracted files
!ls

### Step 2: Importing Required Libraries

In [ ]:
# Import necessary libraries for data handling and processing
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image
from sklearn.model_selection import train_test_split

### Step 3: Load and Explore Dataset

In [ ]:
# List all files in the 'with_mask' directory
with_mask_files = os.listdir('/content/data/with_mask')
print(with_mask_files[0:5])  # Display first 5 file names
print(with_mask_files[-5:])  # Display last 5 file names

# List all files in the 'without_mask' directory
without_mask_files = os.listdir('/content/data/without_mask')
print(without_mask_files[0:5])
print(without_mask_files[-5:])

# Print the number of images in each category
print('Number of with mask images:', len(with_mask_files))
print('Number of without mask images:', len(without_mask_files))

### Step 4: Assign Labels to Images
- **With Mask**: Label as `1`
- **Without Mask**: Label as `0`

In [ ]:
# Create a label list for the two classes
with_mask_labels = [1] * len(with_mask_files)  # Label all 'with mask' images as 1
without_mask_labels = [0] * len(without_mask_files)  # Label all 'without mask' images as 0

# Combine both label lists
labels = with_mask_labels + without_mask_labels

# Display label information
print(len(labels))  # Total number of labels
print(labels[0:5])  # First 5 labels
print(labels[-5:])  # Last 5 labels

### Step 5: Visualize Sample Images

In [ ]:
# Display a sample image from the 'with mask' category
img = mpimg.imread('/content/data/with_mask/with_mask_1545.jpg')
plt.imshow(img)
plt.title('With Mask')
plt.show()

# Display a sample image from the 'without mask' category
img = mpimg.imread('/content/data/without_mask/without_mask_2925.jpg')
plt.imshow(img)
plt.title('Without Mask')
plt.show()

### Step 6: Image Preprocessing
- Resize all images to 128x128.
- Convert to RGB format.
- Convert images to numpy arrays.

In [ ]:
# Directory path for images
with_mask_path = '/content/data/with_mask/'
without_mask_path = '/content/data/without_mask/'

data = []  # List to store processed image data

# Process 'with mask' images
for img_file in with_mask_files:
    image = Image.open(with_mask_path + img_file)  # Open image
    image = image.resize((128, 128))  # Resize to 128x128
    image = image.convert('RGB')  # Convert to RGB
    image = np.array(image)  # Convert to numpy array
    data.append(image)

# Process 'without mask' images
for img_file in without_mask_files:
    image = Image.open(without_mask_path + img_file)
    image = image.resize((128, 128))
    image = image.convert('RGB')
    image = np.array(image)
    data.append(image)

# Convert data and labels to numpy arrays
X = np.array(data)  # Image data
Y = np.array(labels)  # Corresponding labels

# Verify the shapes of the data arrays
print(X.shape, Y.shape)

### Step 7: Train-Test Split
- Split the data into training and testing sets.
- Normalize the pixel values by scaling them between 0 and 1.

In [ ]:
# Split the dataset into training (80%) and testing (20%) sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

# Normalize pixel values by dividing by 255
X_train_scaled = X_train / 255.0
X_test_scaled = X_test / 255.0

### Step 8: Build and Train the CNN Model

In [ ]:
# Import TensorFlow for building the model
import tensorflow as tf
from tensorflow import keras

# Define the CNN model architecture
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),  # First convolutional layer
    keras.layers.MaxPooling2D(2, 2),  # First pooling layer
    keras.layers.Conv2D(64, (3, 3), activation='relu'),  # Second convolutional layer
    keras.layers.MaxPooling2D(2, 2),  # Second pooling layer
    keras.layers.Flatten(),  # Flatten the feature maps
    keras.layers.Dense(128, activation='relu'),  # Fully connected layer
    keras.layers.Dropout(0.5),  # Dropout to prevent overfitting
    keras.layers.Dense(2, activation='softmax')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with the training data
history = model.fit(X_train_scaled, Y_train, validation_split=0.1, epochs=5)

### Step 9: Evaluate the Model

In [ ]:
# Evaluate the model performance on the test data
loss, accuracy = model.evaluate(X_test_scaled, Y_test)
print('Test Accuracy:', accuracy)

# Plot the training and validation loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss over Epochs')
plt.show()

# Plot the training and validation accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy over Epochs')
plt.show()